# EDA: data understanding, visual and correlation analysis

Целью данной лекции будет продолжение изучения практических основ разведочного анализа с помощью библиотеки Pandas --- группировка данных, подсчёт, визуализация, работа с пропущенными значениями. Также мы познакомимся с анализом корреляций между признаками и статистическими тестами.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();

In [ ]:
df = pd.read_csv('../input/cardio_train.csv', sep=';', index_col=0)

In [ ]:
df.head().T

In [ ]:
df.info()

## Типы признаков: непрерывные, порядковые и категориальные

Результаты применения метода info() показали, что данный датасет содержит только числовые признаки. Однако это не должно вводить исследователя в заблуждение: часто категориальные признаки кодируются числами. Более того: для построения моделей машинного обучения **все** признаки должны быть числовыми.

Например, такие признаки, как **gender**, **smoke**, **alco**, **active** и целевой признак **cardio** являются **категориальными**. В таблице значения этих признаков кодируются числами, однако на множествах этих чисел не имеют смысла отношения порядка (больше, меньше, больше или равно, меньше или равно).

Признаки **cholesterol** и **gluc** относятся к **порядковым** признакам, т. е. условным обозначениям категорий, для которых имеет смысл отношение порядка. Например, значение **cholesterol=1** означает **более низкий** уровень холестерина, чем **cholesterol=2**.

Все остальные признаки могут трактоваться как **числовые непрерывные** (несмотря на то, что некоторые из них представлены только целыми числами).

## Анализ корреляций числовых признаков

Выделим числовые признаки в отдельную группу и рассмотрим некоторые простейшие виды корреляционного анализа таких данных.

In [ ]:
df['age_years'] = df['age'] / 365.25 # возраст в годах
numeric = ['age_years', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc']
sns.pairplot(df[numeric]);

Вспомним, что в данных есть выбросы --- явно некорректные значения, которые будут мешать дальнейшей работе. Удалим их.

In [ ]:
sns.boxplot(df['age_years']);

In [ ]:
sns.boxplot(df['height']);

In [ ]:
sns.boxplot(df['weight']);

In [ ]:
sns.boxplot(df['ap_hi']);

In [ ]:
sns.boxplot(df['ap_lo']);

In [ ]:
def outliers_indices(feature):
    '''
    Будем считать выбросами все точки, выходящие за пределы трёх сигм.
    '''
    mid = df[feature].mean()
    sigma = df[feature].std()
    return df[(df[feature] < mid - 3*sigma) | (df[feature] > mid + 3*sigma)].index

In [ ]:
wrong_height = outliers_indices('height')
wrong_weight = outliers_indices('weight')
wrong_hi = outliers_indices('ap_hi')
wrong_lo = outliers_indices('ap_lo')

out = set(wrong_height) | set(wrong_weight) | set(wrong_hi) | set(wrong_lo)

print(len(out))

Из набора данных будет удалено 1929 объектов-выбросов, что не является существенным в данном случае.

In [ ]:
df.drop(out, inplace=True)

Построим матрицу корреляций Спирмена для числовых признаков без учёта объектов-выбросов.

In [ ]:
sns.pairplot(df[numeric]);

In [ ]:
df[numeric].corr(method='spearman')

Функция [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) библиотеки Seaborn предоставляет удобный способ визуализации таких матриц.

In [ ]:
sns.heatmap(df[numeric].corr(method='spearman'));

Какие выводы можно сделать? Наибольшая корреляция наблюдается между переменными **ap_hi** и **ap_lo**. Достаточно высокой является корреляция между ростом и весом, а также уровнями холестерина и глюкозы. Кроме того, можно заметить, что вес в большей степени связан с давлением, чем рост.

## Статистические тесты

Выше было сказано, что корреляция между ростом и весом является _достаточно высокой_, а _вес больше связан с давлением, чем рост_. Пришло время придать олее строгую краску нашим выводам. Поговорим о **статистической значимости** выявленных взаимосвязей. Иными словами, **насколько высокой** должна быть степень корреляции, чтобы можно было вообще говорить о какой-либо **неслучайной взаимосвязи** между признаками?

### Модуль stats библиотеки SciPy

Модуль [stats](https://docs.scipy.org/doc/scipy/reference/stats.html) библиотеки SciPy содержит несколько встроенных функций для вычисления коэффициентов корреляции между признаками различной природы, а также проверки их на статистическую значимость. 

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = pearsonr(df['height'], df['weight'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

In [ ]:
r = pearsonr(df['cholesterol'], df['weight'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

In [ ]:
r = spearmanr(df['cholesterol'], df['weight'])
print('Spearman correlation:', r[0], 'p-value:', r[1])

Так как p-value < 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) между ростом и весом **статистически значима**.

Проверим, действительно ли корреляция роста и давления статистически незначима в отличие от корреляции веса и давления. Снова воспользуемся коэффициентом корреляции Пирсона.

In [ ]:
r_height_pressure = pearsonr(df['height'], df['ap_hi'])
print('Height vs. Pressure:', r_height_pressure)
r_weight_pressure = pearsonr(df['weight'], df['ap_hi'])
print('Weight vs. Pressure:', r_weight_pressure)

Как видим, p-значения в обоих случаях достаточно малы, поэтому у нас **нет оснований** утверждать, что между ростом и давлением нет **статистически подтверждённой взаимосвязи**. Тот факт, что значение $r=0.01$ оказалось значимым, может быть обусловлен большой длиной выборки. Однако в любом случае корреляция давления и роста существенно меньше, чем корреляция давления и веса. 

## Анализ корреляций категориальных признаков

Корреляция между категориальными переменными не может быть измерена с помощью коэффициентов Пирсона, Спирмена и Кендалла. Коэффициенты и выводы для категориальных данных обычно строятся на основании **таблиц сопряжённости** (кросс-таблиц, contingency tables).

Рассмотрим взаимосвязь курения и сердечно-сосудистых заболеваний. Построим кросс-таблицу.

In [ ]:
pd.crosstab(df['smoke'], df['cardio'])

In [ ]:
sns.heatmap(pd.crosstab(df['smoke'], df['cardio']), 
            cmap="YlGnBu", annot=True, cbar=False);

На этом этапе сложно утверждать что-либо. Попробуем вычислить коэффициент $\chi^2$.

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['smoke'], df['cardio']))

Малое значение p-value говорит о том, что связь статистически подтверждается. В данном случае у курящих людей склонность с ССЗ ниже. Этот же вывод подтверждает тест Фишера.

In [ ]:
fisher_exact(pd.crosstab(df['smoke'], df['cardio']))

Для таблиц сопряжённости $2\times 2$ также существуют и другие показатели степени тесноты статистической связи: коэффициент ассоциации, коэффициент контингенции, коэффициент коллигации, коэффициент Гудмена--Краскала.

## Взаимосвязь категориального и числового признаков

В случае бинарного категориального признака и числового признака мы можем применить бисериальный коэффициент корреляции. Например, рассмотрим взаимосвязь веса и сердечно-сосудистых заболеваний.

In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['cardio'], df['weight'])

И снова связь подтвердилась.

## Pairwise Correlation cheat sheet
Шпаргалку по корреляционному анализу признаков различных типов можно скачать [тут](https://gdurl.com/Huca).